In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

c:\Users\Usuario\anaconda3\envs\ML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Definir la arquitectura de la CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #Canales de entrada, Canales de salida, Tamaño de Kernel, Stride y Padding
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.conv2 = nn.Conv2d(16, 64, 3, 1, 1) 
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1) 
        self.fc1 = nn.Linear(128 * 4 * 4, 512) 
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        #Entrada (batch, channels, ancho, largo)
        # (64, 3, 32, 32)
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2) # No es una capa aprendible. (feature_maps, stride, padding)
        # (64, 16, 16, 16)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        # (64, 64, 8, 8)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2, 2)
        # (64, 128, 4, 4)
        x = x.view(-1, 128 * 4 * 4)
        # (64, 2048)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:


# Descargar y cargar el conjunto de datos CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(45),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#Iterable del dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#Iterable del dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is Torch Cuda Available? " + str(torch.cuda.is_available()))

Is Torch Cuda Available? False


In [ ]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = CNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test images: %d %%' % (100 * correct / total))